In [2]:
run = 'one'
bun = 'one'

In [1]:
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
#from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import pandas as pd
from xml.dom import minidom
#from tqdm import tqdm
from PIL import Image, ImageDraw
from os import listdir
from os.path import isfile, join
import random
from math import floor
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib import patches

ModuleNotFoundError: No module named 'cv2'

In [2]:
class ImageAnnotationPreProccesing:

    def __init__(self, images_path, annotations_path):
        self.image_path = images_path
        self.annotations_path = annotations_path
        self.image_files = [self.image_path + '/' + file for file in listdir(self.image_path) if '.jpg' in file] #the image files
        ann_files = [self.annotations_path + '/' + file for file in listdir(self.annotations_path) if '.xml' in file] #the annotation files


    def train_test(self):
        random.seed(2873)
        #grab the file names without the jpg or the xml
        file_prefixes = [self.image_path + '/' + file.split('.jpg')[0] for file in listdir(self.image_path) if '.jpg' in file]
        #shuffle modifies the original list so we crerate a copy
        random.shuffle(file_prefixes)

        #We'll be doing a 70% Train test and splir
        split = 0.7
        split_index = floor(len(file_prefixes) * split)
        training = file_prefixes[:split_index]
        testing = file_prefixes[split_index:]


    def create_test_or_trainig_folder(self, test_or_train, destination_folder):
        path_img = []
        path_ann = []

        for prefix in test_or_train:
            full_path_img = prefix + '.jpg'
            full_path_ann = prefix + '.xml'
            #uncomment line down below if this is the first time your creating a training and testing cat
            #shutil.copy(full_path_img, destination_folder) 
            path_img.append(full_path_img)
            path_ann.append(full_path_ann)

        return path_img, path_ann
    
    # Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):


    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []
    info_dict['file_name'] = xml_file

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                if subelem.text != None:
                    image_size.append(int(subelem.text))

            #since this is black and white we only have one band at the end
            image_size.append(1)
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(float(subsubelem.text))      

                if subelem.tag == "name":
                    bbox["class"] = subelem.text    
                bbox['img_name'] = xml_file.split('.')[0] + '.jpg' #gets the jpg name

            info_dict['bboxes'].append(bbox)


    final_frame = pd.DataFrame(info_dict['bboxes'])

    return final_frame


def get_all_annotations(ann_files):
    '''
    Iterate through all of the annoation files and create a bounding box 

    '''

    final_frame = pd.DataFrame()

    for file in ann_files:
        ann_info = extract_info_from_xml(file)
        final_frame = pd.concat([final_frame, ann_info], ignore_index = True)

    return final_frame[['img_name', 'xmin', 'ymin','xmax', 'ymax', 'class']]
        

    